In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3" 
from tensorflow.python.client import device_lib
print device_lib.list_local_devices()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16901026280551559432
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11983277261
locality {
  bus_id: 1
}
incarnation: 12329227431647653142
physical_device_desc: "device: 0, name: TITAN X (Pascal), pci bus id: 0000:05:00.0, compute capability: 6.1"
]


In [2]:
import tensorflow as tf
import numpy as np
from StochasticDropoutNet_unrolled import StochasticDropoutNet
from util import xor_data, extract_update_dict
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=False)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# model = StochasticDropoutNet(min_dropout_rate = 0.001, max_dropout_rate = 0.001, num_weight_train_steps = 64)
model = StochasticDropoutNet(min_init_dropout_rate = 0.4, 
                             max_init_dropout_rate = 0.6,
                             train_batch_size = 50,
                             valid_batch_size = 500,
                             unroll_steps = 5,
                             num_weight_train_steps = 4,
                             head = 563,
                             log_dir = "/mnt/hdd1/kqian3/rl_struct")

[<tf.Tensor 'StopGradient_6:0' shape=(28, 28, 64) dtype=float32>, <tf.Tensor 'StopGradient_7:0' shape=(28, 28, 64) dtype=float32>, <tf.Tensor 'StopGradient_8:0' shape=(14, 14, 128) dtype=float32>, <tf.Tensor 'StopGradient_9:0' shape=(14, 14, 128) dtype=float32>, <tf.Tensor 'StopGradient_10:0' shape=(1, 1, 2048) dtype=float32>, <tf.Tensor 'StopGradient_11:0' shape=(1, 1, 2048) dtype=float32>]


In [4]:
model.saver.restore(model.sess, "{}model_unroll-562".format("/mnt/hdd1/kqian3/rl_struct/"))

INFO:tensorflow:Restoring parameters from /mnt/hdd1/kqian3/rl_struct/model_unroll-562


In [6]:
inputs_train, targets_train = mnist.train.next_batch(32)
inputs_train = inputs_train.reshape((-1, 28, 28, 1))
inputs_valid, targets_valid = mnist.validation.next_batch(32)
inputs_valid = inputs_valid.reshape((-1, 28, 28, 1))
feed_dict = {model.inputs: inputs_train[0 : 32, ...],
             model.targets: targets_train[0 : 32, ...]}
print(model.sess.run([model.valid_index, tf.gradients(model.loss, model.struct_param)], feed_dict = feed_dict))


[[array([[[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 0., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 0., 1., 1.],
        [1., 0., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [0., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[1., 1., 0., ..., 1., 0., 1.],
        [1., 1., 1., ..., 1., 0., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       ...,

       [[1., 1., 1., ..., 0., 0., 1.],
        [1., 1., 0., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 0., 1., 1.],
        [1., 1., 1., ..., 1., 

In [ ]:
inputs_train, targets_train = mnist.train.next_batch(55000)
inputs_train = inputs_train.reshape((-1, 28, 28, 1))
inputs_valid, targets_valid = mnist.validation.next_batch(5000)
inputs_valid = inputs_valid.reshape((-1, 28, 28, 1))

model.train_unroll(inputs_train, targets_train, inputs_valid, targets_valid, num_epochs = 401)
#model.train_fine_tune(inputs_train, targets_train, inputs_valid, targets_valid, num_epochs = 20)

In [ ]:
inputs_test, targets_test = mnist.test.next_batch(10000)
inputs_test = inputs_test.reshape((-1, 28, 28, 1))
accs = []
for i in range(10):
    feed_dict = {model.inputs: inputs_test[i*1000:(i+1)*1000, ...], 
                 model.targets: targets_test[i*1000:(i+1)*1000, ...]}
    acc = model.sess.run(model.accuracy, feed_dict=feed_dict)
    accs.append(acc)
    
print(sum(accs)/10)

In [ ]:
import csv
with open('./log/results.csv', 'a+') as f:
    writer = csv.writer(f)
    writer.writerow(model.train_losses)
    writer.writerow(model.valid_losses)

In [ ]:
feed_dict = {model.inputs: inputs_train[0 : 32, ...],
             model.targets: targets_train[0 : 32, ...]}
print(model.pred)
print(model.sess.run(model.pred, feed_dict = feed_dict))
print(model.sess.run(model.targets, feed_dict = feed_dict))
print(model.struct_train_op)

In [ ]:
inputs_train, targets_train = mnist.train.next_batch(55000)
a = (targets_train.shape)
print(a)